---
# **Nornir**
---

### **Библиотека ```Nornir```, работа с файлами**

Библиотека ```Nornir``` (далее ```Nornir```) является реализацией фреймворка по автоматизации настройки сетевого оборудования, целиком написанной на языке Python. Данный факт позиционируется как преимущество по сравнению с другими фреймворками, так как весь программный код открыт и отладка работы как скриптов, так и самих модулей библиотеки происходит исключительно используя инструменты Python. Для начала работы с ```Nornir``` необходимо установить библиотеку с помощью pip и импортировать необходимые модули. Одним из ключевых компонентов ```Nornir``` является библиотека ```NAPALM```, которая включает в себя инструменты для взаимодействия по API с набором популярного сетевого оборудования (Cisco, Juniper, Arista), а также библиотека ```Netmiko```, которая используется для взаимодействия с сетевыми устройствами посредством протокола SSH.

In [27]:
import pyeapi

def patch_pyeapi_ciphers():
    """
    python -c 'import ssl; print(ssl._DEFAULT_CIPHERS)'
    @SECLEVEL=2:ECDH+AESGCM:ECDH+CHACHA20:ECDH+AES:DHE+AES:!aNULL:!eNULL:!aDSS:!SHA1:!AESCCM
    """
    try:
        import pyeapi.eapilib
    except ImportError:
        return

    connect_orig = pyeapi.eapilib.HttpsConnection.connect

    def connect(self):
        self._context.set_ciphers('DEFAULT@SECLEVEL=2')
        return connect_orig(self)

    pyeapi.eapilib.HttpsConnection.connect = connect


patch_pyeapi_ciphers()

In [1]:
!pip3 install nornir

Defaulting to user installation because normal site-packages is not writeable
  Using cached nornir-3.3.0-py3-none-any.whl (30 kB)
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
nornir-napalm 0.4.0 requires napalm<5,>=4, which is not installed.


---
Для выполнения базовых операций достаточно импортировать компонент ```nornir.core```.

In [2]:
import nornir.core

---
1.2 Инициализация ```Nornir``` осуществляется с помощью создания экземпляра класса ```InitNornir``` и подачи конструктору в качестве аргумента параметров, которые могут быть заданы в виде YAML файла ```config.yaml```.

In [ ]:
!cat config.yaml

In [5]:
from nornir import InitNornir

In [6]:
nr = InitNornir(config_file='config.yaml')

In [ ]:
nr = InitNornir(
    runner={
        "plugin": "threaded",
        "options": {
            "num_workers": 4,
        },
    },
    inventory={
        "plugin": "SimpleInventory",
        "options": {
            "host_file": "inventory/hosts.yaml",
            "group_file": "inventory/groups.yaml",
            "defaults_file": "inventory/defaults.yaml"
        },
    },
)

In [15]:
nr.inventory.hosts['Leaf2'].extended_data()

{'rack': 1,
 'role': 'leaf',
 'id': 2,
 'domain': 'spb.company.ru',
 'purpose': 'lab'}

In [19]:
id_2 = nr.filter(id=2)

In [20]:
id_2.inventory.hosts

{'Leaf2': Host: Leaf2}

In [21]:
id_2.inventory.hosts['Leaf2'].extended_data()

{'rack': 1,
 'role': 'leaf',
 'id': 2,
 'domain': 'spb.company.ru',
 'purpose': 'lab'}

In [ ]:
print(dir(nr))

In [ ]:
print(dir(nr.inventory))

In [ ]:
print(dir(nr.inventory.hosts))

In [ ]:
print(dir(nr.inventory.defaults))

In [ ]:
nr.inventory.defaults.password = 'admin'  #input() или getpass()

---
Файл ```hosts.yaml``` описывает основные параметры подконтрольных сетевых устройств.

In [ ]:
!cat inventory/hosts.yaml

---
Файл ```groups.yaml``` содержит информацию, которая может описывать общие параметры для устройств в файле ```hosts.yaml```.

In [ ]:
!cat inventory/groups.yaml

---
Файл ```defaults.yaml``` содержит информацию, которая является параметрами по умолчанию для устройств в файле ```hosts.yaml```.

In [ ]:
!cat inventory/defaults.yaml

---
Информацию по устройствам можно вывести на экран или в файл, вызывая атрибуты объекта ```inventory```

In [ ]:
print(nr.inventory.hosts)

In [ ]:
print(nr.inventory.groups)

---
Выбрать устройство для дальнейшего применения действий можно, обратившись к `inventory.hosts` как к словарю.

In [ ]:
Leaf1 = nr.inventory.hosts['Leaf1']

print(Leaf1.groups)

In [ ]:
Leaf1.platform

In [ ]:
print(Leaf1['purpose'])

In [ ]:
print(dir(Leaf1))

In [ ]:
print(Leaf1.hostname, Leaf1.name, Leaf1.password)

---
```Nornir``` имеет встроенные инструменты фильтрации данных из файлов. Встроенный метод ```filter``` позволяет описать критерии фильтрации.

In [ ]:
spb_switches = nr.filter(domain='spb.company.ru').inventory.hosts
spb_switches

In [ ]:
nr.filter(role='leaf').inventory.hosts.keys()

---
Метод ```children_of_group()``` позволяет запросить участников определенной группы.

In [ ]:
nr.inventory.children_of_group('spb')

---
---

### **Библиотека ```Nornir```, конфигурация**

---
```Nornir``` имеет возможность применять изменения конфигурации к устройствам и выполнять отладочные команды. Для выполнения базовых команд на сетевом оборудовании дополнительно импортируется модуль ```commands```. Для вывода результатов используется модуль ```print_result```.

In [22]:
!pip install nornir_netmiko

Defaulting to user installation because normal site-packages is not writeable
  Using cached netmiko-4.1.2-py3-none-any.whl (196 kB)
  Attempting uninstall: netmiko
    Found existing installation: netmiko 4.2.0
    Uninstalling netmiko-4.2.0:
      Successfully uninstalled netmiko-4.2.0


In [23]:
!pip install nornir_utils

Defaulting to user installation because normal site-packages is not writeable


---
С помощью фильтров данных на основании файлов ```inventory``` выбирается группа устройств, к которым должны быть применены команды. Для выполнения команды создается экземпляр класса ```DEVICES.run()```, конструктору которого подается задание на выполнение (```task```) и команды. Функция ```print_result``` используется для полного или частичного вывода информации в поток ввода/вывода.

In [24]:
from nornir import InitNornir
from nornir_netmiko import netmiko_send_command
from nornir_utils.plugins.functions import print_result

nr = InitNornir(config_file='config.yaml')

result = nr.run(task=netmiko_send_command, command_string='show arp')

print_result(result)

netmiko_send_command************************************************************
* Leaf1 ** changed : False *****************************************************
vvvv netmiko_send_command ** changed : False vvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvv INFO
Address         Age (sec)  Hardware Addr   Interface
172.20.20.1       0:00:00  0242.7464.27e1  Management0
^^^^ END netmiko_send_command ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
* Leaf2 ** changed : False *****************************************************
vvvv netmiko_send_command ** changed : False vvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvv INFO
Address         Age (sec)  Hardware Addr   Interface
172.20.20.1       0:00:00  0242.7464.27e1  Management0
^^^^ END netmiko_send_command ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^


In [ ]:
result

In [ ]:
result.items()

In [ ]:
result['Leaf1']

In [ ]:
result['Leaf1'][0]

In [ ]:
result['Leaf1'][0].result

---
Для сбора основной информации с устройств импортируется модуль ```networking```. В составе библиотеки ```Nornir``` содержится библиотека ```NAPALM```, в которой реализованы вспомогательные операции по взаимодействию с сетевыми устройствами. В частности имеется инструмент ```getters```, который может собирать 'факты' (```"facts"```) об устройствах.

In [25]:
!pip install nornir_napalm

Defaulting to user installation because normal site-packages is not writeable


In [29]:
from nornir import InitNornir
from nornir_utils.plugins.functions import print_result
from nornir_napalm.plugins.tasks import napalm_get

nr = InitNornir(config_file='config.yaml')

leaf_hosts = nr.filter(role="leaf")

result = leaf_hosts.run(task=napalm_get, getters=["facts", "lldp_neighbors"])

print_result(result)

napalm_get**********************************************************************
* Leaf1 ** changed : False *****************************************************
vvvv napalm_get ** changed : False vvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvv INFO
{ 'facts': { 'fqdn': 'Leaf1.msk.company.ru',
             'hostname': 'Leaf1',
             'interface_list': ['Ethernet1', 'Loopback100', 'Management0'],
             'model': 'cEOSLab',
             'os_version': '4.28.3M-28837868.4283M',
             'serial_number': '73AE505E46FD5F8A965B00DEEBC3C3A7',
             'uptime': 12116.20351934433,
             'vendor': 'Arista'},
  'lldp_neighbors': { 'Ethernet1': [ { 'hostname': 'Leaf2.spb.company.ru',
                                       'port': 'Ethernet1'}],
                      'Management0': [ { 'hostname': 'Leaf2.spb.company.ru',
                                         'port': 'Management0'}]}}
^^^^ END napalm_get ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
* Leaf2

In [ ]:
print(dir(result['Leaf1'][0]))

In [ ]:
dict_res = result['Leaf1'][0].result

pprint(dict_res)

---
Задание (```task```) является базовым классом библиотеки ```Nornir```. Наследуя данный класс, имеется возможность конструировать пользовательские задания, одиночные или комплексные. Далее, под именем объекта ```task``` они могут быть вызваны в конструкторе ```run()```.

In [ ]:
from nornir import InitNornir
from nornir_utils.plugins.functions import print_result
from nornir_napalm.plugins.tasks import napalm_get

def available_resources(device):
    r1 = device.run(task=netmiko_send_command,
             name="UPTIME",
             command_string="show version | i Uptime")
    print_result(r1)
    r2 = device.run(task=netmiko_send_command,
             name="VERSION",
             command_string="show version | i version")
    print_result(r2)
    
nr = InitNornir(config_file='config.yaml')

leaf_hosts = nr.filter(role="leaf")
    
leaf_hosts.run(task=available_resources)

---
Для выполнения конфигурационных изменений допускается конструирование файлов-шаблонов команд, применяемых к устройствам. В составе ```Nornir``` используется библиотека ```Jinja2```, которая позволяет создавать и обрабатывать шаблоны текстовых данных. Используя данные шаблоны генерируются последовательности команд, в которые подставляются значения из набора ```inventory```. Для этого из библиотеки ```Nornir``` импортируется модуль ```text```.

In [ ]:
!pip install nornir_jinja2

In [ ]:
from nornir_utils.plugins.functions import print_result, print_title
from nornir_napalm.plugins.tasks import napalm_configure
from nornir_jinja2.plugins.tasks import template_file

---
Для проверки возможности изменения конфигурации и точности полученных на основе шаблонов именений используется вспомогательный ключ ```dry_run=True```. Данный ключ позволяет осуществить имитацию применения изменения конфигурации (на платформах, поддерживающих понятие ```candidate_config``` конфигурация применяется в данных контейнер; на остальных устройствах содаются вспомогательные директории, куда может быть помещен файл изменений). Тем не менее, рекомендуется, по возможности, осуществлять отладку комплексных изменений конфигурации на тестовом оборудовании. Ключ ```dry_run``` может быть изменен в рамках конкретного задания, для всех устройств или выборочно для отдельных устройств. Для этого модифицируется аттрибут ```nr.data.dry_run = <True/False>``` или аттрибут результата фильтрации ```<RESULT>.data.dry_run = <True/False>```.

In [ ]:
nr = InitNornir(config_file="config.yaml")

leafs = nr.filter(role="leaf")

print(leafs.inventory.hosts.keys())

---
При создании пользовательского задания в рамках конструирования экземпляра класса ```run()``` по ключу ```template``` подается шаблон ```Jinja2```, в котором описаны конфигурационные команды.

In [ ]:
!cat templates/eos/base.j2

In [28]:
from nornir_utils.plugins.functions import print_result, print_title
from nornir_napalm.plugins.tasks import napalm_configure
from nornir_jinja2.plugins.tasks import template_file

nr = InitNornir(config_file="config.yaml")

leafs = nr.filter(role="leaf")


def basic_configuration(device):
    
    r1 = device.run(task=template_file,
                 name="Base Configuration",
                 template="base.j2",
                 path=f"templates/{device.host.platform}")

    r2 = device.run(task=napalm_configure,
             name="Loading Configuration on the device",
             replace=False,
             configuration=r1.result)
    
    print_result(r2)
    

print_title("Playbook to configure the network")

leafs.run(task=basic_configuration)

**** Playbook to configure the network *****************************************


AggregatedResult (basic_configuration): {'Leaf1': MultiResult: [Result: "basic_configuration", Result: "Base Configuration", Result: "Loading Configuration on the device"], 'Leaf2': MultiResult: [Result: "basic_configuration", Result: "Base Configuration", Result: "Loading Configuration on the device"]}

---
Для применения изменений к устройствам ключ ```dry_run``` переводится в значение ```False```.

---
Пример просмотра конфигурации устройств.

In [ ]:
from nornir import InitNornir
from nornir_napalm.plugins.tasks import napalm_get
from nornir_utils.plugins.functions import print_result

nr = InitNornir(config_file='config.yaml')

leaf1 = nr.filter(name='Leaf1')

result = leaf1.run(task=napalm_get, getters=['config'])

print_result(result)

---
Пример бэкапирования конфигурации набора устройств.

In [30]:
from nornir import InitNornir
from nornir_utils.plugins.tasks.files import write_file
from nornir_napalm.plugins.tasks import napalm_get

def backup(device, path):
    r = device.run(
        task=napalm_get,
        getters=["config"]
    )
    device.run(
        task=write_file,
        filename=f"{path}/{device.host}",
        content=r.result["config"]["running"],
    )

nr = InitNornir(config_file='config.yaml', dry_run=False)

result = nr.run(name='Backup configuration',
                task=backup,
                path='backups')